# 02. Extract team statistics

### Imports

In [14]:
import pandas as pd
import numpy as np

### Load processed training dataset

In [16]:
training_set = pd.read_parquet("datasets/processed/training_set_processed.parquet")

### Extract all teams in training set

In [17]:
home_teams_list = training_set["HT"].unique().tolist()
away_teams_list = training_set["AT"].unique().tolist()
teams_list = list(set(home_teams_list + away_teams_list))
print("Number of teams in training set:",len(teams_list))

Number of teams in training set: 1637


In [18]:
teams_list

['Barcelona SC',
 'YoungHeart Manawatu',
 'Ironi Ramat HaSharon',
 'Icasa',
 'Albacete',
 'Virtus Lanciano',
 'FK Rostov',
 'Napoli',
 'ES Metlaoui',
 'Perugia',
 'Kataller Toyama',
 'Valladolid',
 'Tianjin Teda',
 'Droylsden',
 'Sevilla FC B',
 'Confianca',
 'DRB Tadjenanet',
 'Degerfors',
 'FK Ural',
 'Villarreal B',
 'Tokushima Vortis',
 'Mouloudia Oujda',
 'Wolfsburg',
 'Roasso Kumamoto',
 'Nublense',
 'Gainare Tottori',
 'Chemnitzer FC',
 'IF Elfsborg',
 'Jahn Regensburg',
 'Toulouse',
 'Dundee United',
 'Portland Timbers',
 'Vejle',
 'Macclesfield',
 'Tampa Bay Rowdies',
 'FK Khimki',
 'KAC de Kenitra',
 'Alloa Athletic',
 'Coquimbo Unido',
 'Penafiel',
 'Dumbarton',
 'Mainz 05 II',
 'Perth Glory',
 'Jokerit Helsinki',
 'Seongnam FC',
 'Granada 74',
 'Fluminense Rio de Janeiro',
 'Kashima Antlers',
 'TP47 Tornio',
 'Rosenborg Trondheim',
 'Sport Club Recife',
 'Maccabi Netanya',
 'Eastbourne Borough',
 'Zweigen',
 'Kotkan Tyovaen Palloilijat',
 'FC Superfund',
 'Sao Bento',
 'Pen

### Functions to gather each team statistics 

In [19]:
def extract_team_df(team_name):
    ''' Extracts games of a specific team from whole training set.
    '''
    
    home_team_df = training_set[training_set["HT"] == team_name]
    away_team_df = training_set[training_set["AT"] == team_name]
        
    team_df = pd.concat([home_team_df, away_team_df]).sort_values(by="Date")
    
    return team_df, home_team_df, away_team_df

In [20]:
def get_stats(df, local=True):
    '''Fills in team statistics dictionary.
    '''
    team_stats = dict()
    if local is True:
        prefix = "home"
        goals_scored_col = "HS"
        goals_against_col = "AS"
        win, loss = "W", "L"
        goals_difference = df["GD"].sum()
        try:
            avg_goals_difference = round(df["GD"].sum() / df.shape[0], 3)
        except ZeroDivisionError:
            avg_goals_difference = np.nan
    else:
        prefix = "away"
        goals_scored_col = "AS"
        goals_against_col = "HS"
        win, loss = "L", "W"
        goals_difference = - df["GD"].sum()
        try:
            avg_goals_difference = - round(df["GD"].sum() / df.shape[0], 3)
        except ZeroDivisionError:
            avg_goals_difference = np.nan

    team_stats[f"{prefix}_games"] = df.shape[0]
    
    team_stats[f"{prefix}_wins"] = df["WDL"].value_counts().get(win, 0)
    team_stats[f"{prefix}_draws"] = df["WDL"].value_counts().get("D", 0)
    team_stats[f"{prefix}_losses"] = df["WDL"].value_counts().get(loss, 0)
    
    try:
        team_stats[f"{prefix}_win_percentage"] = round(df["WDL"].value_counts().get(win, 0) / df.shape[0], 3)
    except ZeroDivisionError:
        team_stats[f"{prefix}_win_percentage"] = np.nan
    try:
        team_stats[f"{prefix}_draw_percentage"] = round(df["WDL"].value_counts().get("D", 0) / df.shape[0], 3)
    except ZeroDivisionError:
        team_stats[f"{prefix}_draw_percentage"] = np.nan
    try:
        team_stats[f"{prefix}_loss_percentage"] = round(df["WDL"].value_counts().get(loss, 0) / df.shape[0], 3)
    except ZeroDivisionError:
        team_stats[f"{prefix}_loss_percentage"] = np.nan
    team_stats[f"{prefix}_goals_scored"] = df[goals_scored_col].sum()
    team_stats[f"{prefix}_goals_against"] = df[goals_against_col].sum()
    team_stats[f"{prefix}_goals_difference"] = goals_difference
    try:
        team_stats[f"{prefix}_avg_goals_scored"] = round(df[goals_scored_col].sum() / df.shape[0], 3)
    except ZeroDivisionError:
        team_stats[f"{prefix}_avg_goals_scored"] = np.nan
    try:
        team_stats[f"{prefix}_avg_goals_against"] = round(df[goals_against_col].sum() / df.shape[0], 3)
    except ZeroDivisionError:
        team_stats[f"{prefix}_avg_goals_against"] = np.nan
    team_stats[f"{prefix}_avg_goals_difference"] = avg_goals_difference

    return team_stats

### Basic function to fill in each team dict with specific data

In [21]:
def complete_team_info(team_name):

    # Filter games only of specified team
    team_df, home_team_df, away_team_df = extract_team_df(team_name)
        
    # Create team dict
    team_stats = dict()

    # Recover home and away teams separately
    home_stats = get_stats(home_team_df, local=True)
    away_stats = get_stats(away_team_df, local=False)

    # Add home and away stats
    team_stats_temp = home_stats | away_stats

    # Add general stats
    team_stats["name"] = team_name
    team_stats["total_games"] = team_df.shape[0]
    team_stats["wins"] = team_stats_temp["home_wins"] + team_stats_temp["away_wins"]
    team_stats["draws"] = team_stats_temp["home_draws"] + team_stats_temp["away_draws"]
    team_stats["losses"] = team_stats_temp["home_losses"] + team_stats_temp["away_losses"]
    team_stats["win_percentage"] = round(team_stats["wins"] / team_df.shape[0], 3)
    team_stats["draw_percentage"] = round(team_stats["draws"] / team_df.shape[0], 3)
    team_stats["loss_percentage"] = round(team_stats["losses"] / team_df.shape[0], 3)
    team_stats["goals_scored"] = team_stats_temp["home_goals_scored"] + team_stats_temp["away_goals_scored"]
    team_stats["goals_against"] = team_stats_temp["home_goals_against"] + team_stats_temp["away_goals_against"]
    team_stats["goals_difference"] = team_stats["goals_scored"] - team_stats["goals_against"]
    team_stats["avg_goals_scored"] = round(team_stats["goals_scored"] / team_df.shape[0], 3)
    team_stats["avg_goals_against"] = round(team_stats["goals_against"] / team_df.shape[0], 3)
    team_stats["avg_goals_difference"] = round(team_stats["goals_difference"] / team_df.shape[0], 3)
    
    team_stats = team_stats | home_stats | away_stats
       
    return team_stats

### Complete each team info 

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [24]:
teams = list()

# For each team, complete its info...
for team in teams_list:
    team_stats = complete_team_info(team_name=team)
    teams.append(team_stats)

### Check list with all teams dicts info isolated

In [26]:
teams

# with open("teams.pickle") as teams_file:
#    pickle.dump(teams, teams_file)

[{'name': 'Barcelona SC',
  'total_games': 410,
  'wins': 191,
  'draws': 106,
  'losses': 113,
  'win_percentage': 0.466,
  'draw_percentage': 0.259,
  'loss_percentage': 0.276,
  'goals_scored': 557,
  'goals_against': 392,
  'goals_difference': 165,
  'avg_goals_scored': 1.359,
  'avg_goals_against': 0.956,
  'avg_goals_difference': 0.402,
  'home_games': 205,
  'home_wins': 124,
  'home_draws': 47,
  'home_losses': 34,
  'home_win_percentage': 0.605,
  'home_draw_percentage': 0.229,
  'home_loss_percentage': 0.166,
  'home_goals_scored': 346,
  'home_goals_against': 145,
  'home_goals_difference': 201,
  'home_avg_goals_scored': 1.688,
  'home_avg_goals_against': 0.707,
  'home_avg_goals_difference': 0.98,
  'away_games': 205,
  'away_wins': 67,
  'away_draws': 59,
  'away_losses': 79,
  'away_win_percentage': 0.327,
  'away_draw_percentage': 0.288,
  'away_loss_percentage': 0.385,
  'away_goals_scored': 211,
  'away_goals_against': 247,
  'away_goals_difference': -36,
  'away_avg_

### Function to search H2H games (team1 vs team2)

In [36]:
def search_H2H(team1, team2):
    
    condition_a = ((training_set["HT"] == team1) & (training_set["AT"] == team2))
    condition_b = ((training_set["HT"] == team2) & (training_set["AT"] == team1))
    
    h2h = training_set.loc[condition_a | condition_b]
    
    return h2h

In [37]:
search_H2H("Real Madrid", "Eibar")

,Sea,Lge,Date,HT,AT,HS,AS,GD,WDL,Goals
174450,14-15,SPA1,2014-11-22,Eibar,Real Madrid,0,4,-4,L,4
174640,14-15,SPA1,2015-04-11,Real Madrid,Eibar,3,0,3,W,3
189935,15-16,SPA1,2015-11-29,Eibar,Real Madrid,0,2,-2,L,2
190122,15-16,SPA1,2016-04-09,Real Madrid,Eibar,4,0,4,W,4
212361,16-17,SPA1,2016-10-02,Real Madrid,Eibar,1,1,0,D,2
212545,16-17,SPA1,2017-03-04,Eibar,Real Madrid,1,4,-3,L,5
219409,17-18,SPA1,2017-10-22,Real Madrid,Eibar,3,0,3,W,3
219593,17-18,SPA1,2018-03-10,Eibar,Real Madrid,1,2,-1,L,3
224852,18-19,SPA1,2018-11-24,Eibar,Real Madrid,3,0,3,W,3
225032,18-19,SPA1,2019-04-06,Real Madrid,Eibar,2,1,1,W,3


### Function to get last N games of specific team

In [34]:
def get_last_matches(team_name, n_previous_matches):

    team_df, _, _= extract_team_df(team_name)
    last_n_games_df = team_df[-n_previous_matches:]

    return last_n_games_df

In [35]:
get_last_matches("Real Madrid", 900)

,Sea,Lge,Date,HT,AT,HS,AS,GD,WDL,Goals
4104,00-01,SPA1,2000-09-09,Real Madrid,Valencia,2,1,1,W,3
4113,00-01,SPA1,2000-09-16,Malaga,Real Madrid,3,3,0,D,6
4124,00-01,SPA1,2000-09-23,Real Madrid,Athletic Bilbao,4,1,3,W,5
4132,00-01,SPA1,2000-09-30,Santander,Real Madrid,0,0,0,D,0
4143,00-01,SPA1,2000-10-14,Real Madrid,La Coruna,3,0,3,W,3
...,...,...,...,...,...,...,...,...,...,...
298599,22-23,SPA1,2023-01-07,Villarreal,Real Madrid,2,1,1,W,3
298623,22-23,SPA1,2023-01-22,Athletic Bilbao,Real Madrid,0,2,-2,L,2
298633,22-23,SPA1,2023-01-29,Real Madrid,Real Sociedad,0,0,0,D,0
298636,22-23,SPA1,2023-02-02,Real Madrid,Valencia,2,0,2,W,2


------------------

### **Load teams file from pickle**

In [ ]:
with open("teams.pickle", "rb") as teams_file:
    teams = pickle.load(teams_file)